In [1]:
import keras.preprocessing.image
import keras.utils
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from keras.models import  model_from_json,Sequential
from keras.models import Model as Md
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *
from attention import Attention

#from skimage import color
#from sklearn.metrics import accuracy_score
#import cv2b

import numpy as np
import os
from keras.layers import Flatten
import pandas as pd

In [2]:
def load_fsdd(spectrograms_path):
    x_train = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram)
    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    return x_train



In [3]:
def load_meta(meta_csv):
    cols = list(pd.read_csv(meta_csv, nrows =1))
    meta = pd.read_csv(meta_csv, usecols =[i for i in cols if i != 'clip_name'])
    y_train = np.array(meta)          
    
    return y_train

In [4]:
x_train = load_fsdd('C:/Users/user/Documents/data sci class/Untitled Folder/train_ss')


In [5]:
np.shape(x_train)

(44, 200, 101, 1)

In [6]:
y_train = load_meta('C:/Users/user/Documents/data sci class/Untitled Folder/train6.csv')

In [7]:
np.shape(y_train)

(44, 8)

In [8]:
images = keras.layers.Input(x_train.shape[1:])
images

<KerasTensor: shape=(None, 200, 101, 1) dtype=float32 (created by layer 'input_1')>

In [28]:
def Model():
    
# ResNet block 1a
    
    images = keras.layers.Input(x_train.shape[1:])
    
    x = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(images)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2a
    block2 = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3a
    block3 = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    #inio Squeeze and Excitation 1a
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,16))(sq)
    sq = keras.layers.Dense(units=16,activation="relu")(sq)
    sq = keras.layers.Dense(units=16,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(images)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)


    #SE-ResNet block 2

    # ResNet block 1b
    x = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2b
    block2 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3b
    block3 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    #Squeeze and Excitation 1b
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,32))(sq)
    sq = keras.layers.Dense(units=32,activation="relu")(sq)
    sq = keras.layers.Dense(units=32,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)




    #SE-ResNet block 3

    # ResNet block 1c
    x = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2c
    block2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3c
    block3 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    #Squeeze and Excitation 1c
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,64))(sq)
    sq = keras.layers.Dense(units=64,activation="relu")(sq)
    sq = keras.layers.Dense(units=64,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)


    #SE-ResNet block 4

    # ResNet block 1d
    x = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2d
    block2 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3d
    block3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    # Squeeze and Excitation 1d
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,128))(sq)
    sq = keras.layers.Dense(units=128,activation="relu")(sq)
    sq = keras.layers.Dense(units=128,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)
    
    
    
    #SE-ResNet block 5

    # ResNet block 1e
    x = keras.layers.Conv2D(filters=256, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2e
    block2 = keras.layers.Conv2D(filters=256, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3e
    block3 = keras.layers.Conv2D(filters=256, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    # Squeeze and Excitation 1e
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,256))(sq)
    sq = keras.layers.Dense(units=256,activation="relu")(sq)
    sq = keras.layers.Dense(units=256,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=256, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)



    
    #SE-ResNet block 6

    # ResNet block 1f
    x = keras.layers.Conv2D(filters=512, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2f
    block2 = keras.layers.Conv2D(filters=512, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3f
    block3 = keras.layers.Conv2D(filters=512, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    # Squeeze and Excitation 1f
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,512))(sq)
    sq = keras.layers.Dense(units=512,activation="relu")(sq)
    sq = keras.layers.Dense(units=512,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=512, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)
    
    
    #SE-ResNet block 7

    # ResNet block 1g
    x = keras.layers.Conv2D(filters=1024, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2g
    block2 = keras.layers.Conv2D(filters=1024, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3g
    block3 = keras.layers.Conv2D(filters=1024, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    # Squeeze and Excitation 1g
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,1024))(sq)
    sq = keras.layers.Dense(units=1024,activation="relu")(sq)
    sq = keras.layers.Dense(units=1024,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=1024, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)
    
    
    
    
    #SE-ResNet block 8

    # ResNet block 1h
    x = keras.layers.Conv2D(filters=2048, kernel_size=[3, 3], padding='same')(net)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2h
    block2 = keras.layers.Conv2D(filters=2048, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3h
    block3 = keras.layers.Conv2D(filters=2048, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    # Squeeze and Excitation 1h
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,2048))(sq)
    sq = keras.layers.Dense(units=2048,activation="relu")(sq)
    sq = keras.layers.Dense(units=2048,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=2048, kernel_size=[3, 3], padding='same')(net)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)

    #net = Flatten()(net)
    

    #BLSTM
    #forwards = LSTM(2048,return_sequences=True, activation='sigmoid' )(net)
    #backwards = LSTM(2048,return_sequences=True,activation='sigmoid', go_backwards = True)(net)
    #merged = merge([forwards,backwards],mode='mul')
    #after_dp = Dropout(0.2)(merged)
    
    
    
    #Output layer, Attention Network
    #net = Attention(units=512)(net)
    net = Flatten()(net)
    net = Dropout(0.5)(net) 
    x = Dense(8)(net)
    net = keras.layers.Activation("relu")(x)
    
    model = keras.models.Model(inputs=images,outputs=net)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [26]:

def train(x_trin, y_trin, learning_rate = 0.7, batch_size =int(11) , epochs=int(4)):
    #print(model.summary())
    #checkpointer = keras.callbacks.ModelCheckpoint(filepath = 'cnn_from_scratch_fruits.hdf5', verbose = 1, save_best_only = True)
    #earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
    #train_model = Model().fit(x_train, y_train, learning_rate, batch_size, epochs,  callbacks = [checkpointer,earlystopper], shuffle=True)
    train_model = Model().fit(x_trin, y_trin, learning_rate, batch_size, epochs)
    #train_model.load_weights('SE-ResNet_for_disfluency.hdf5')
    return train_model

In [ ]:
Model().fit(x_train, y_train, batch_size=11, epochs=4)

Epoch 1/4


In [24]:
Model().summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 101, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_33 (Conv2D)             (None, 200, 101, 16  160         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_33 (BatchN  (None, 200, 101, 16  64         ['conv2d_33[0][0]']              
 ormalization)                  )                                                           

                                )                                                                 
                                                                                                  
 global_average_pooling2d_9 (Gl  (None, 32)          0           ['activation_39[0][0]']          
 obalAveragePooling2D)                                                                            
                                                                                                  
 reshape_9 (Reshape)            (None, 1, 1, 32)     0           ['global_average_pooling2d_9[0][0
                                                                 ]']                              
                                                                                                  
 dense_20 (Dense)               (None, 1, 1, 32)     1056        ['reshape_9[0][0]']              
                                                                                                  
 conv2d_40

 ormalization)                  8)                                                                
                                                                                                  
 activation_45 (Activation)     (None, 200, 101, 12  0           ['batch_normalization_45[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_46 (Conv2D)             (None, 200, 101, 12  147584      ['activation_45[0][0]']          
                                8)                                                                
                                                                                                  
 batch_normalization_46 (BatchN  (None, 200, 101, 12  512        ['conv2d_46[0][0]']              
 ormalization)                  8)                                                                
          

 conv2d_52 (Conv2D)             (None, 200, 101, 25  295168      ['activation_48[0][0]']          
                                6)                                                                
                                                                                                  
 dense_27 (Dense)               (None, 1, 1, 256)    65792       ['dense_26[0][0]']               
                                                                                                  
 batch_normalization_52 (BatchN  (None, 200, 101, 25  1024       ['conv2d_52[0][0]']              
 ormalization)                  6)                                                                
                                                                                                  
 multiply_12 (Multiply)         (None, 200, 101, 25  0           ['activation_51[0][0]',          
                                6)                                'dense_27[0][0]']               
          

                                                                                                  
 activation_58 (Activation)     (None, 200, 101, 10  0           ['batch_normalization_58[0][0]'] 
                                24)                                                               
                                                                                                  
 conv2d_59 (Conv2D)             (None, 200, 101, 10  9438208     ['activation_58[0][0]']          
                                24)                                                               
                                                                                                  
 batch_normalization_59 (BatchN  (None, 200, 101, 10  4096       ['conv2d_59[0][0]']              
 ormalization)                  24)                                                               
                                                                                                  
 activatio

                                                                                                  
 add_15 (Add)                   (None, 200, 101, 20  0           ['batch_normalization_64[0][0]', 
                                48)                               'multiply_15[0][0]']            
                                                                                                  
 activation_64 (Activation)     (None, 200, 101, 20  0           ['add_15[0][0]']                 
                                48)                                                               
                                                                                                  
 flatten_1 (Flatten)            (None, 41369600)     0           ['activation_64[0][0]']          
                                                                                                  
 dense_35 (Dense)               (None, 8)            330956808   ['flatten_1[0][0]']              
          

In [1]:
Model().load_weights('SE-ResNet_for_disfluency.hdf5')

train_model = model.fit(x_train, y_train, batch_size, epochs,
                      validation_data=(x_val, y_val), callbacks = [checkpointer,earlystopper], shuffle=True)




NameError: name 'Model' is not defined

In [ ]:
y_test_pred = model.predict(x_test)
accuracy_score(np.argmax(y_test_pred,axis=1), np.argmax(y_test,axis=1))